# Probabilistic Time Series Forecasting with 🤗 Transformers

## Introduction

Time series forecasting is an essential scientific and business problem and as such has also seen a lot of innovation recently with the use of [deep learning based](https://dl.acm.org/doi/abs/10.1145/3533382) models in addition to the [classical methods](https://otexts.com/fpp3/). An important difference between classical methods like ARIMA and novel deep learning methods is the following.

##  Probabilistic Forecasting

Typically, classical methods are fitted on each time series in a dataset individually. These are often referred to as  "single" or "local" methods. However, when dealing with a large amount of time series for some applications, it is beneficial to train a "global" model on all available time series, which enables the model to learn latent representations from many different sources.

Some classical methods are point-valued (meaning, they just output a single value per time step) and models are trained by minimizing an L2 or L1 type of loss with respect to the ground truth data. However, since forecasts are often used in some real-world decision making pipeline, even with humans in the loop, it is much more beneficial to provide the uncertainties of predictions. This is also called "probabilistic forecasting", as opposed to "point forecasting". This entails modeling a probabilistic distribution, from which one can sample.

So in short, rather than training local point forecasting models, we hope to train **global probabilistic** models. Deep learning is a great fit for this, as neural networks can learn representations from several related time series as well as model the uncertainty of the data.

It is common in the probabilistic setting to learn the future parameters of some chosen parametric distribution, like Gaussian or Student-T; or learn the conditional quantile function; or use the framework of Conformal Prediction adapted to the time series setting. The choice of method does not affect the modeling aspect and thus can be typically thought of as yet another hyperparameter. One can always turn a probabilistic model into a point-forecasting model, by taking empirical means or medians.

## The Time Series Transformer

In terms of modeling time series data which are sequential in nature, as one can imagine, researchers have come up with models which use Recurrent Neural Networks (RNN) like LSTM or GRU, or Convolutional Networks (CNN), and more recently Transformer based methods which fit naturally to the time series forecasting setting.

In this blog post, we're going to leverage the vanilla Transformer [(Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762) for the **univariate** probabilistic forecasting task (i.e. predicting each time series' 1-d distribution individually). The Encoder-Decoder Transformer is a natural choice for forecasting as it encapsulates several inductive biases nicely.

To begin with, the use of an Encoder-Decoder architecture is helpful at inference time where typically for some logged data we wish to forecast some prediction steps into the future. This can be thought of as analogous to the text generation task where given some context, we sample the next token and pass it back into the decoder (also called "autoregressive generation"). Similarly here we can also, given some distribution type, sample from it to provide forecasts up until our desired prediction horizon. This is known as Greedy Sampling/Search and there is a great blog post about it [here](https://huggingface.co/blog/how-to-generate) for the NLP setting.

Secondly, a Transformer helps us to train on time series data which might contain thousands of time points. It might not be feasible to input *all* the history of a time series at once to the model, due to the time- and memory constraints of the attention mechanism. Thus, one can consider some appropriate context window and sample this window and the subsequent prediction length sized window from the training data when constructing batches for stochastic gradient descent (SGD). The context sized window can be passed to the encoder and the prediction window to a *causal-masked* decoder. This means that the decoder can only look at previous time steps when learning the next value. This is equivalent to how one would train a vanilla Transformer for machine translation, referred to as "teacher forcing".

Another benefit of Transformers over the other architectures is that we can incorporate missing values (which are common in the time series setting) as an additional mask to the encoder or decoder and still train without resorting to in-filling or imputation. This is equivalent to the `attention_mask` of models like BERT and GPT-2 in the Transformers library, to not include padding tokens in the computation of the attention matrix.

A drawback of the Transformer architecture is the limit to the sizes of the context and prediction windows because of the quadratic compute and memory requirements of the vanilla Transformer, see [Tay et al., 2020](https://arxiv.org/abs/2009.06732). Additionally, since the Transformer is a powerful architecture, it might overfit or learn spurious correlations much more easily compared to other [methods](https://openreview.net/pdf?id=D7YBmfX_VQy).

The 🤗 Transformers library comes with a vanilla probabilistic time series Transformer model, simply called the [Time Series Transformer](https://huggingface.co/docs/transformers/model_doc/time_series_transformer). In the sections below, we'll show how to train such a model on a custom dataset.

## Set-up Environment

First, let's install the necessary libraries: 🤗 Transformers, 🤗 Datasets, 🤗 Evaluate,  🤗 Accelerate and [GluonTS](https://github.com/awslabs/gluonts).

As we will show, GluonTS will be used for transforming the data to create features as well as for creating appropriate training, validation and test batches.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install -q transformers

In [3]:
# !pip install -q datasets

In [4]:
!pip install -q evaluate

In [5]:
!pip install -q accelerate

In [6]:
!pip install -q gluonts ujson

In [7]:
!pip install -q lightning

In [8]:
!pip install -q scipy

In [9]:
!pip install -q matplotlib

In [10]:
!pip install -q scikit-learn

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [11]:
# from transformers.utils import send_example_telemetry

# send_example_telemetry("time_series_transformers_notebook", framework="pytorch")

In [2]:
import numpy as np
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split, InputDataset, LabelDataset

In [3]:
from gluonts.time_feature import (
    time_features_from_frequency_str,
    TimeFeature,
    get_lags_for_frequency,
)
from gluonts.dataset.field_names import FieldName
from gluonts.transform import (
    AddAgeFeature,
    AddObservedValuesIndicator,
    AddTimeFeatures,
    AsNumpyArray,
    Chain,
    ExpectedNumInstanceSampler,
    InstanceSplitter,
    RemoveFields,
    SelectFields,
    SetField,
    TestSplitSampler,
    Transformation,
    ValidationSplitSampler,
    VstackFeatures,
    RenameFields,
)

In [4]:
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction

In [5]:
import matplotlib.pyplot as plt

In [6]:
import random
import time

In [7]:
from data_preprocessor.data_preprocessor import CompositeDataPreprocessor
from data_preprocessor.feature_engineering import EnrichDFDataPreprocessor, MovingAvgPreProcessor, RemoveIrrelevantFeaturesDataPreprocessor, DropTargetNADataPreprocessor

In [8]:
np.random.seed(0)
random.seed(0)

In [9]:
from evaluate import load
from gluonts.time_feature import get_seasonality

mae_metric = load("evaluate-metric/mae")

In [10]:
import sys

In [11]:
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

## Load Dataset

In this blog post, we'll use the `tourism_monthly` dataset, which is available on the [Hugging Face Hub](https://huggingface.co/datasets/monash_tsf). This dataset contains monthly tourism volumes for 366 regions in Australia.

This dataset is part of the [Monash Time Series Forecasting](https://forecastingdata.org/) repository, a collection of  time series datasets from a number of domains. It can be viewed as the GLUE benchmark of time series forecasting.

In [12]:
df = pd.read_csv(
    "../optiver-trading-at-the-close/train.csv",
    dtype={
        "seconds_in_bucket": np.float32,
        "imbalance_size": np.float32,
        "imbalance_buy_sell_flag": np.float32,
        "reference_price": np.float32,
        "matched_size": np.float32,
        "far_price": np.float32,
        "near_price": np.float32,
        "bid_price": np.float32,
        "bid_size": np.float32,
        "ask_price": np.float32,
        "ask_size": np.float32,
        "wap": np.float32,
        "target": np.float32,
        "time_id": np.float32,
    },
)
raw_df = df

In [13]:
raw_df.dtypes

stock_id                     int64
date_id                      int64
seconds_in_bucket          float32
imbalance_size             float32
imbalance_buy_sell_flag    float32
reference_price            float32
matched_size               float32
far_price                  float32
near_price                 float32
bid_price                  float32
bid_size                   float32
ask_price                  float32
ask_size                   float32
wap                        float32
target                     float32
time_id                    float32
row_id                      object
dtype: object

In [65]:
df = raw_df

In [66]:
processors = [
    EnrichDFDataPreprocessor(),
    MovingAvgPreProcessor("wap"),
    # DropTargetNADataPreprocessor(),
    # RemoveIrrelevantFeaturesDataPreprocessor(['stock_id', 'date_id','time_id', 'row_id']),
]
processor = CompositeDataPreprocessor(processors)

In [67]:
df = processor.apply(df)

In [68]:
print(df.shape[0])
print(df.columns)
print(df.dtypes)
display(df)

5237980
Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id',
       'timestamp_by_time_id', 'timestamp', 'imb_s1', 'imb_s2',
       'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2', 'ask_price_bid_price_wap_imb2',
       'pressure', 'inefficiency', 'wap_mov_avg_3_1', 'wap_mov_avg_6_3',
       'wap_mov_avg_12_6', 'wap_mov_avg_24_12'],
      dtype='object')
stock_id                                        int64
date_id                                         int64
seconds_in_bucket                             float32
imbalance_size                                float32
imbalance_buy_sell_flag                       float32
reference_price                               float32
matched_size        

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,reference_price_ask_price_bid_price_imb2,reference_price_ask_price_wap_imb2,reference_price_bid_price_wap_imb2,ask_price_bid_price_wap_imb2,pressure,inefficiency,wap_mov_avg_3_1,wap_mov_avg_6_3,wap_mov_avg_12_6,wap_mov_avg_24_12
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00,0,0,0.0,3.180603e+06,1.0,0.999812,13380277.00,NaN,NaN,0.999812,...,NaN,0.139683,NaN,0.139683,374.495636,0.237708,1.000000,NaN,NaN,NaN
2018-01-01 00:00,1,0,0.0,1.666039e+05,-1.0,0.999896,1642214.25,NaN,NaN,0.999896,...,12816.000000,6.344985,1744.000000,6.344985,51.531654,0.101451,1.000000,NaN,NaN,NaN
2018-01-01 00:00,2,0,0.0,3.028799e+05,-1.0,0.999561,1819368.00,NaN,NaN,0.999403,...,4.662142,0.678887,2.776772,0.499201,7.979763,0.166475,1.000000,NaN,NaN,NaN
2018-01-01 00:00,3,0,0.0,1.191768e+07,-1.0,1.000171,18389746.00,NaN,NaN,0.999999,...,0.251127,0.252617,168.705887,239.466660,5126.105469,0.648061,1.000000,NaN,NaN,NaN
2018-01-01 00:00,4,0,0.0,4.475500e+05,-1.0,0.999532,17860614.00,NaN,NaN,0.999394,...,3.507559,0.034131,3.391793,0.026360,27.148033,0.025058,1.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-19 08:54,195,480,540.0,2.440723e+06,-1.0,1.000317,28280362.00,0.999734,0.999734,1.000317,...,NaN,9.673913,-47.000000,9.673913,75.664818,0.086305,1.000615,1.000617,1.000778,1.000732
2018-01-19 08:54,196,480,540.0,3.495105e+05,-1.0,1.000643,9187699.00,1.000129,1.000386,1.000643,...,NaN,0.460705,NaN,0.460705,1.704028,0.038041,0.999665,0.999360,0.999172,0.998990
2018-01-19 08:54,197,480,540.0,0.000000e+00,0.0,0.995789,12725436.00,0.995789,0.995789,0.995789,...,1576.000000,10.857142,NaN,10.857142,0.000000,0.000000,1.001383,1.001192,1.001166,1.001111


In [69]:
max_time_id = df["time_id"].max()
dti_by_time_id = pd.date_range("2018-01-01", periods=max_time_id + 1, freq="min")
print(dti_by_time_id)
df_period_index_by_time_id = dti_by_time_id.to_period("1min")
print(df_period_index_by_time_id)
df_index_by_time_id_series = df_period_index_by_time_id.to_series(index=np.arange(max_time_id + 1))
print(df_index_by_time_id_series)

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 00:01:00',
               '2018-01-01 00:02:00', '2018-01-01 00:03:00',
               '2018-01-01 00:04:00', '2018-01-01 00:05:00',
               '2018-01-01 00:06:00', '2018-01-01 00:07:00',
               '2018-01-01 00:08:00', '2018-01-01 00:09:00',
               ...
               '2018-01-19 08:45:00', '2018-01-19 08:46:00',
               '2018-01-19 08:47:00', '2018-01-19 08:48:00',
               '2018-01-19 08:49:00', '2018-01-19 08:50:00',
               '2018-01-19 08:51:00', '2018-01-19 08:52:00',
               '2018-01-19 08:53:00', '2018-01-19 08:54:00'],
              dtype='datetime64[ns]', length=26455, freq='T')
PeriodIndex(['2018-01-01 00:00', '2018-01-01 00:01', '2018-01-01 00:02',
             '2018-01-01 00:03', '2018-01-01 00:04', '2018-01-01 00:05',
             '2018-01-01 00:06', '2018-01-01 00:07', '2018-01-01 00:08',
             '2018-01-01 00:09',
             ...
             '2018-01-19 08:45', '2018-

In [70]:
df["timestamp_by_time_id"] = df["time_id"].map(df_index_by_time_id_series)

In [71]:
df["timestamp"] = df["timestamp_by_time_id"]
df.index = df["timestamp"]

In [72]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,reference_price_ask_price_bid_price_imb2,reference_price_ask_price_wap_imb2,reference_price_bid_price_wap_imb2,ask_price_bid_price_wap_imb2,pressure,inefficiency,wap_mov_avg_3_1,wap_mov_avg_6_3,wap_mov_avg_12_6,wap_mov_avg_24_12
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00,0,0,0.0,3.180603e+06,1.0,0.999812,13380277.00,NaN,NaN,0.999812,...,NaN,0.139683,NaN,0.139683,374.495636,0.237708,1.0,NaN,NaN,NaN
2018-01-01 00:00,1,0,0.0,1.666039e+05,-1.0,0.999896,1642214.25,NaN,NaN,0.999896,...,12816.000000,6.344985,1744.000000,6.344985,51.531654,0.101451,1.0,NaN,NaN,NaN
2018-01-01 00:00,2,0,0.0,3.028799e+05,-1.0,0.999561,1819368.00,NaN,NaN,0.999403,...,4.662142,0.678887,2.776772,0.499201,7.979763,0.166475,1.0,NaN,NaN,NaN
2018-01-01 00:00,3,0,0.0,1.191768e+07,-1.0,1.000171,18389746.00,NaN,NaN,0.999999,...,0.251127,0.252617,168.705887,239.466660,5126.105469,0.648061,1.0,NaN,NaN,NaN
2018-01-01 00:00,4,0,0.0,4.475500e+05,-1.0,0.999532,17860614.00,NaN,NaN,0.999394,...,3.507559,0.034131,3.391793,0.026360,27.148033,0.025058,1.0,NaN,NaN,NaN


In [73]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 5237980 entries, 2018-01-01 00:00 to 2018-01-19 08:54
Freq: T
Data columns (total 31 columns):
 #   Column                                    Dtype    
---  ------                                    -----    
 0   stock_id                                  int64    
 1   date_id                                   int64    
 2   seconds_in_bucket                         float32  
 3   imbalance_size                            float32  
 4   imbalance_buy_sell_flag                   float32  
 5   reference_price                           float32  
 6   matched_size                              float32  
 7   far_price                                 float32  
 8   near_price                                float32  
 9   bid_price                                 float32  
 10  bid_size                                  float32  
 11  ask_price                                 float32  
 12  ask_size                                  float32  

In [74]:
df_grouped = df.groupby("stock_id")
print(len(df_grouped))
print(df_grouped.size())

200
stock_id
0      26455
1      26455
2      26455
3      26455
4      26455
       ...  
195    26455
196    26455
197    26455
198    26455
199    21615
Length: 200, dtype: int64


In [75]:
dfs_dict = {}
for item_id, gdf in df_grouped:
    dfs_dict[item_id] = gdf.reindex(df_index_by_time_id_series).drop("stock_id", axis=1)

In [76]:
dfs_dict[0]

,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,...,reference_price_ask_price_bid_price_imb2,reference_price_ask_price_wap_imb2,reference_price_bid_price_wap_imb2,ask_price_bid_price_wap_imb2,pressure,inefficiency,wap_mov_avg_3_1,wap_mov_avg_6_3,wap_mov_avg_12_6,wap_mov_avg_24_12
2018-01-01 00:00,0,0.0,3.180603e+06,1.0,0.999812,13380277.0,NaN,NaN,0.999812,60651.500000,...,NaN,0.139683,NaN,0.139683,374.495636,0.237708,1.000000,NaN,NaN,NaN
2018-01-01 00:01,0,10.0,1.299773e+06,1.0,1.000026,15261107.0,NaN,NaN,0.999812,13996.500000,...,0.0,0.000000,1.671131,1.671131,55.264420,0.085169,1.000008,NaN,NaN,NaN
2018-01-01 00:02,0,20.0,1.299773e+06,1.0,0.999919,15261107.0,NaN,NaN,0.999812,4665.500000,...,0.0,-0.001546,2.568588,2.568588,107.139435,0.085169,1.000282,1.000282,NaN,NaN
2018-01-01 00:03,0,30.0,1.299773e+06,1.0,1.000133,15261107.0,NaN,NaN,1.000026,55998.000000,...,0.0,0.000000,0.810484,0.810484,28.131599,0.085169,0.998740,0.999055,NaN,NaN
2018-01-01 00:04,0,40.0,1.218204e+06,1.0,1.000455,15342675.0,NaN,NaN,1.000241,14655.950195,...,0.0,0.001730,1.809077,1.809077,45.779175,0.079400,1.000396,1.000218,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-19 08:50,480,500.0,0.000000e+00,0.0,0.999017,42161928.0,0.999017,0.999017,0.999017,53827.199219,...,-2954.0,2.677459,-806.000000,2.677459,0.000000,0.000000,0.998857,0.998951,0.999076,0.999062
2018-01-19 08:51,480,510.0,0.000000e+00,0.0,0.998842,42161928.0,0.998842,0.998842,0.998842,157865.406250,...,NaN,0.768675,-1662.000000,0.768675,0.000000,0.000000,0.999055,0.999029,0.998995,0.998865
2018-01-19 08:52,480,520.0,4.755137e+05,-1.0,0.999193,41686416.0,0.999017,0.999017,0.999193,57596.671875,...,1467.0,0.823602,NaN,0.823602,8.255923,0.011407,0.998500,0.998565,0.998541,0.998846
2018-01-19 08:53,480,530.0,4.755137e+05,-1.0,0.999193,41686416.0,0.999017,0.999017,0.999193,156610.531250,...,1467.0,0.783718,821.000000,0.783718,3.036282,0.011407,0.999762,0.999483,0.999174,0.998948


In [77]:
for item_id, gdf in dfs_dict.items():
    gdf[gdf.columns.difference(["target"])] = gdf[gdf.columns.difference(["target"])].fillna(0.0)

In [78]:
freq = "1min"

In [79]:
df.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id',
       'timestamp_by_time_id', 'timestamp', 'imb_s1', 'imb_s2',
       'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2', 'ask_price_bid_price_wap_imb2',
       'pressure', 'inefficiency', 'wap_mov_avg_3_1', 'wap_mov_avg_6_3',
       'wap_mov_avg_12_6', 'wap_mov_avg_24_12'],
      dtype='object')

In [80]:
feat_dynamic_real = [column for column in df.columns if column not in [
    "stock_id",
    "date_id",
    "target",
    # "time_id",
    "row_id",
    "timestamp_by_time_id",
    "timestamp"
]]
print(len(feat_dynamic_real), feat_dynamic_real)

25 ['seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'time_id', 'imb_s1', 'imb_s2', 'reference_price_ask_price_bid_price_imb2', 'reference_price_ask_price_wap_imb2', 'reference_price_bid_price_wap_imb2', 'ask_price_bid_price_wap_imb2', 'pressure', 'inefficiency', 'wap_mov_avg_3_1', 'wap_mov_avg_6_3', 'wap_mov_avg_12_6', 'wap_mov_avg_24_12']


In [81]:
# feat_dynamic_real = ["imbalance_size", "reference_price", "matched_size"]
# feat_dynamic_real = []

In [82]:
dataset = PandasDataset(dfs_dict, target="target", feat_dynamic_real=feat_dynamic_real, freq=freq, assume_sorted=False)
print(dataset)
print(len(dataset))

PandasDataset<size=200, freq=1min, num_feat_dynamic_real=25, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>
200


In [83]:
prediction_length = 1

In [162]:
validation_length = 55 * 20
# window will not overlap
validation_window_size = int(validation_length / prediction_length)

# Split the data for training and testing
training_data, test_gen = split(dataset, offset=-validation_length)
test_data = test_gen.generate_instances(prediction_length=prediction_length, windows=validation_window_size)

val_data, _ = split(dataset, offset=-1)

In [163]:
train_dataset = training_data
test_data_input_dataset = InputDataset(test_data)
test_data_label_dataset = LabelDataset(test_data)
print(len(train_dataset), len(test_data_input_dataset), len(test_data_label_dataset))
print(train_dataset)
print(test_data_input_dataset)
print(test_data_label_dataset)

train_dataset_iter = iter(training_data)
test_data_input_dataset_iter = iter(test_data_input_dataset)
test_data_label_dataset_iter = iter(test_data_label_dataset)

200 220000 220000
TrainingDataset(dataset=PandasDataset<size=200, freq=1min, num_feat_dynamic_real=25, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>, splitter=OffsetSplitter(offset=-1100))
InputDataset(test_data=TestData(dataset=PandasDataset<size=200, freq=1min, num_feat_dynamic_real=25, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>, splitter=OffsetSplitter(offset=-1100), prediction_length=1, windows=1100, distance=None, max_history=None))
LabelDataset(test_data=TestData(dataset=PandasDataset<size=200, freq=1min, num_feat_dynamic_real=25, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>, splitter=OffsetSplitter(offset=-1100), prediction_length=1, windows=1100, distance=None, max_history=None))


In [223]:
print(len(val_data))
print(val_data)
print(next(iter(val_data))["target"].shape)

200
TrainingDataset(dataset=PandasDataset<size=200, freq=1min, num_feat_dynamic_real=25, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>, splitter=OffsetSplitter(offset=-1))
(26454,)


In [166]:
def print_data_sample(sample):
    print(f"sample snippet: {sample}")
    print(f"sample dict keys: {sample.keys()}")
    print(f'start: {sample["start"]} {type(sample["start"])}')
    print(f'target shape: {type(sample["target"])} {sample["target"].shape}')
    print(f'item id: {sample["item_id"]}')
    if "feat_dynamic_real" in sample:
        print(f'feat_dynamic_real": {type(sample["feat_dynamic_real"])} {sample["feat_dynamic_real"].shape} {sample["feat_dynamic_real"].dtype}')
    else:
        print("no feat_dynamic_real")

In [167]:
train_sample = next(train_dataset_iter)
print_data_sample(train_sample)

sample snippet: {'start': Period('2018-01-01 00:00', 'T'), 'target': array([-3.029704  ,  0.38981438,  4.220009  , ..., -4.580021  ,
       -6.740093  ,  0.61035156], dtype=float32), 'item_id': 0, 'feat_dynamic_real': array([[0.0000000e+00, 1.0000000e+01, 2.0000000e+01, ..., 5.2000000e+02,
        5.3000000e+02, 5.4000000e+02],
       [3.1806028e+06, 1.2997728e+06, 1.2997728e+06, ..., 3.3379682e+06,
        3.3363655e+06, 3.3363655e+06],
       [1.0000000e+00, 1.0000000e+00, 1.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 1.0002823e+00, ..., 9.9456370e-01,
        1.0012045e+00, 9.9625033e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 9.9485558e-01,
        1.0012875e+00, 9.9646044e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 9.9579036e-01,
        1.0014405e+00, 9.9649990e-01]], dtype=float32)}
sample dict keys: dict_keys(['start', 'target', 'item_id', 'feat_dynamic_real'])


In [168]:
test_input_sample = next(test_data_input_dataset_iter)
print_data_sample(test_input_sample)
test_label_sample = next(test_data_label_dataset_iter)
print_data_sample(test_label_sample)

print("----------")

test_input_sample = next(test_data_input_dataset_iter)
print_data_sample(test_input_sample)

test_label_sample = next(test_data_label_dataset_iter)
print_data_sample(test_label_sample)

sample snippet: {'start': Period('2018-01-01 00:00', 'T'), 'target': array([-3.029704  ,  0.38981438,  4.220009  , ..., -4.580021  ,
       -6.740093  ,  0.61035156], dtype=float32), 'item_id': 0, 'feat_dynamic_real': array([[0.0000000e+00, 1.0000000e+01, 2.0000000e+01, ..., 5.3000000e+02,
        5.4000000e+02, 0.0000000e+00],
       [3.1806028e+06, 1.2997728e+06, 1.2997728e+06, ..., 3.3363655e+06,
        3.3363655e+06, 1.0247004e+06],
       [1.0000000e+00, 1.0000000e+00, 1.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 1.0002823e+00, ..., 1.0012045e+00,
        9.9625033e-01, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0012875e+00,
        9.9646044e-01, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0014405e+00,
        9.9649990e-01, 0.0000000e+00]], dtype=float32)}
sample dict keys: dict_keys(['start', 'target', 'item_id', 'feat_dynamic_real'])


In [169]:
raw_df[raw_df["stock_id"] == 0].iloc[-validation_length - 2:-validation_length + 2][["target"]]

,target
timestamp,
2018-01-18 14:33,-6.740093
2018-01-18 14:34,0.610352
2018-01-18 14:35,2.900362
2018-01-18 14:36,8.339882


## Define the model

Next, let's instantiate a model. The model will be trained from scratch, hence we won't use the `from_pretrained` method here, but rather randomly initialize the model from a [`config`](https://huggingface.co/docs/transformers/model_doc/time_series_transformer#transformers.TimeSeriesTransformerConfig).

We specify a couple of additional parameters to the model:
- `prediction_length` (in our case, `24` months): this is the horizon that the decoder of the Transformer will learn to predict for;
- `context_length`: the model will set the `context_length` (input of the encoder) equal to the `prediction_length`, if no `context_length` is specified;
- `lags` for a given frequency: these specify how much we "look back", to be added as additional features. e.g. for a `Daily` frequency we might consider a look back of `[1, 2, 7, 30, ...]` or in other words look back 1, 2, ... days while for `Minute` data we might consider `[1, 30, 60, 60*24, ...]` etc.;
- the number of time features: in our case, this will be `2` as we'll add `MonthOfYear` and `Age` features;
- the number of static categorical features: in our case, this will be just `1` as we'll add a single "time series ID" feature;
- the cardinality: the number of values of each static categorical feature, as a list which for our case will be `[366]` as we have 366 different time series
- the embedding dimension: the embedding dimension for each static categorical feature, as a list, for example `[3]` meaning the model will learn an embedding vector of size `3` for each of the `366` time series (regions).


Let's use the default lags provided by GluonTS for the given frequency ("monthly"):

In [170]:
from gluonts.time_feature import get_lags_for_frequency

lags_sequence = get_lags_for_frequency(freq)
print(lags_sequence)

[1, 2, 3, 4, 5, 6, 7, 58, 59, 60, 61, 62, 118, 119, 120, 121, 122, 178, 179, 180, 181, 182]


This means that we'll look back up to 37 months for each time step, as additional features.

Let's also check the default time features which GluonTS provides us:

In [171]:
from gluonts.time_feature import time_features_from_frequency_str

time_features = time_features_from_frequency_str(freq)
print(time_features)

[<function minute_of_hour at 0x150220a879c0>, <function hour_of_day at 0x150220a87b00>, <function day_of_week at 0x150220a87c40>, <function day_of_month at 0x150220a87d80>, <function day_of_year at 0x150220a87ec0>]


In this case, there's only a single feature, namely "month of year". This means that for each time step, we'll add the month as a scalar value (e.g. `1` in case the timestamp is "january", `2` in case the timestamp is "february", etc.).

We now have everything to define the model:

In [172]:
config = TimeSeriesTransformerConfig(
    prediction_length=prediction_length,
    # context length:
    context_length=20,
    # lags coming from helper given the freq:
    # lags_sequence=lags_sequence,
    # we'll add 1 time features ("age", see further):
    num_time_features=0,
    # we have a single static categorical feature, stock ID:
    num_static_categorical_features=1,
    # it has 200 possible values:
    cardinality=[len(df_grouped)],
    # the model will learn an embedding of size 2 for each of the 200 possible values:
    embedding_dimension=[2],
    num_dynamic_real_features=len(feat_dynamic_real),

    # transformer params:
    encoder_layers=4,
    decoder_layers=4,
    d_model=32,
)

model = TimeSeriesTransformerForPrediction(config)

In [173]:
print(config.num_static_categorical_features)
print(config.num_static_real_features)
print(config.num_dynamic_real_features)

1
0
25


Note that, similar to other models in the 🤗 Transformers library, [`TimeSeriesTransformerModel`](https://huggingface.co/docs/transformers/model_doc/time_series_transformer#transformers.TimeSeriesTransformerModel) corresponds to the encoder-decoder Transformer without any head on top, and [`TimeSeriesTransformerForPrediction`](https://huggingface.co/docs/transformers/model_doc/time_series_transformer#transformers.TimeSeriesTransformerForPrediction) corresponds to `TimeSeriesTransformerModel` with a **distribution head** on top. By default, the model uses a Student-t distribution (but this is configurable):

In [174]:
model.config.distribution_output

'student_t'

This is an important difference with Transformers for NLP, where the head typically consists of a fixed categorical distribution implemented as an `nn.Linear` layer.

## Define Transformations

Next, we define the transformations for the data, in particular for the creation of the time features (based on the dataset or universal ones).

Again, we'll use the GluonTS library for this. We define a `Chain` of transformations (which is a bit comparable to `torchvision.transforms.Compose` for images). It allows us to combine several transformations into a single pipeline.

The transformations below are annotated with comments, to explain what they do. At a high level, we will iterate over the individual time series of our dataset and add/remove fields or features:

In [175]:
from transformers import PretrainedConfig


def create_transformation(freq: str, config: PretrainedConfig) -> Transformation:
    remove_field_names = []
    if config.num_static_real_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_REAL)
    if config.num_dynamic_real_features == 0:
        remove_field_names.append(FieldName.FEAT_DYNAMIC_REAL)
    if config.num_static_categorical_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_CAT)

    # a bit like torchvision.transforms.Compose
    return Chain(
        []
        # step 1: remove static/dynamic fields if not specified
        # [RemoveFields(field_names=remove_field_names)]
        # step 2: convert the data to NumPy (potentially not needed)
        + (
            [
                AsNumpyArray(
                    field=FieldName.ITEM_ID,
                    expected_ndim=0,
                    dtype=int,
                )
            ]
            # if config.num_static_categorical_features > 0
            # else []
        )
        + (
            # [
            #     AsNumpyArray(
            #         field=FieldName.FEAT_STATIC_REAL,
            #         expected_ndim=1,
            #     )
            # ]
            # if config.num_static_real_features > 0
            # else []
            []
        )
        + [
            AsNumpyArray(
                field=FieldName.TARGET,
                # we expect an extra dim for the multivariate case:
                expected_ndim=1,
            ),
            # step 3: handle the NaN's by filling in the target with zero
            # and return the mask (which is in the observed values)
            # true for observed values, false for nan's
            # the decoder uses this mask (no loss is incurred for unobserved values)
            # see loss_weights inside the xxxForPrediction model
            AddObservedValuesIndicator(
                target_field=FieldName.TARGET,
                output_field=FieldName.OBSERVED_VALUES,
            ),
            # step 4: add temporal features based on freq of the dataset
            # month of year in the case when freq="M"
            # these serve as positional encodings
            # AddTimeFeatures(
            #     start_field=FieldName.START,
            #     target_field=FieldName.TARGET,
            #     output_field=FieldName.FEAT_TIME,
            #     time_features=time_features_from_frequency_str(freq),
            #     pred_length=config.prediction_length,
            # ),
            # step 5: add another temporal feature (just a single number)
            # tells the model where in the life the value of the time series is
            # sort of running counter
            # AddAgeFeature(
            #     target_field=FieldName.TARGET,
            #     output_field=FieldName.FEAT_AGE,
            #     pred_length=config.prediction_length,
            #     log_scale=False,
            # ),
            # step 6: vertically stack all the temporal features into the key FEAT_TIME
            VstackFeatures(
                output_field=FieldName.FEAT_TIME,
                input_fields=[]
                + (
                    [FieldName.FEAT_DYNAMIC_REAL]
                    if config.num_dynamic_real_features > 0
                    else []
                ),
            ),
            # step 7: rename to match HuggingFace names
            RenameFields(
                mapping={
                    FieldName.ITEM_ID: "static_categorical_features",
                    FieldName.FEAT_STATIC_REAL: "static_real_features",
                    FieldName.FEAT_TIME: "time_features",
                    FieldName.TARGET: "values",
                    FieldName.OBSERVED_VALUES: "observed_mask",
                }
            ),
        ]
    )

## Define `InstanceSplitter`

For training/validation/testing we next create an `InstanceSplitter` which is used to sample windows from the dataset (as, remember, we can't pass the entire history of values to the Transformer due to time- and memory constraints).

The instance splitter samples random `context_length` sized and subsequent `prediction_length` sized windows from the data, and appends a `past_` or `future_` key to any temporal keys in `time_series_fields` for the respective windows. The instance splitter can be configured into three different modes:
1. `mode="train"`: Here we sample the context and prediction length windows randomly from the dataset given to it (the training dataset)
2. `mode="validation"`: Here we sample the very last context length window and prediction window from the dataset given to it (for the back-testing or validation likelihood calculations)
3. `mode="test"`: Here we sample the very last context length window only (for the prediction use case)

In [176]:
from gluonts.transform.sampler import InstanceSampler
from typing import Optional


def create_instance_splitter(
    config: PretrainedConfig,
    mode: str,
    train_sampler: Optional[InstanceSampler] = None,
    validation_sampler: Optional[InstanceSampler] = None,
) -> Transformation:
    assert mode in ["train", "validation", "test"]

    instance_sampler = {
        "train": train_sampler
        or ExpectedNumInstanceSampler(
            num_instances=1.0, min_future=config.prediction_length
        ),
        "validation": validation_sampler
        or ValidationSplitSampler(min_future=config.prediction_length),
        "test": TestSplitSampler(),
    }[mode]

    return InstanceSplitter(
        target_field="values",
        is_pad_field=FieldName.IS_PAD,
        start_field=FieldName.START,
        forecast_start_field=FieldName.FORECAST_START,
        instance_sampler=instance_sampler,
        past_length=config.context_length + max(config.lags_sequence),
        future_length=config.prediction_length,
        time_series_fields=["time_features", "observed_mask"],
    )

## Create DataLoaders

Next, it's time to create the DataLoaders, which allow us to have batches of (input, output pairs) - or in other words (`past_values`, `future_values`).

In [177]:
from typing import Iterable

import torch
from gluonts.itertools import Cyclic, Cached
from gluonts.dataset.loader import as_stacked_batches


def create_train_dataloader(
    config: PretrainedConfig,
    freq,
    data,
    batch_size: int,
    num_batches_per_epoch: int,
    shuffle_buffer_length: Optional[int] = None,
    cache_data: bool = True,
    **kwargs,
) -> Iterable:
    PREDICTION_INPUT_NAMES = [
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    TRAINING_INPUT_NAMES = PREDICTION_INPUT_NAMES + [
        "future_values",
        "future_observed_mask",
    ]

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data, is_train=True)
    if cache_data:
        transformed_data = Cached(transformed_data)

    # we initialize a Training instance
    instance_splitter = create_instance_splitter(config, "train")

    # the instance splitter will sample a window of
    # context length + lags + prediction length (from the 200 possible transformed time series)
    # randomly from within the target time series and return an iterator.
    stream = Cyclic(transformed_data).stream()
    training_instances = instance_splitter.apply(stream)

    return as_stacked_batches(
        training_instances,
        batch_size=batch_size,
        shuffle_buffer_length=shuffle_buffer_length,
        field_names=TRAINING_INPUT_NAMES,
        output_type=torch.tensor,
        num_batches_per_epoch=num_batches_per_epoch,
    )

In [210]:
class CustomBackTestSampler(InstanceSampler):
    back_test_size = 0
    
    def __init__(self, back_test_size):
        super().__init__()
        assert back_test_size > 0
        self.back_test_size = back_test_size

    def __call__(self, ts: np.ndarray) -> np.ndarray:
        data_size = ts.shape[-1]
        # TODO: do not use split offset -1 above
        return np.arange(data_size - self.back_test_size + 1, data_size)

In [211]:
def create_backtest_dataloader(
    config: PretrainedConfig,
    freq,
    data,
    batch_size: int,
    cache_data: bool = True,
    validation_sampler: Optional[InstanceSampler] = None,
    **kwargs,
):
    PREDICTION_INPUT_NAMES = [
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data)
    if cache_data:
        transformed_data = Cached(transformed_data)

    # We create a Validation Instance splitter which will sample the very last
    # context window seen during training only for the encoder.
    instance_sampler = create_instance_splitter(config, "validation", validation_sampler=validation_sampler)

    # we apply the transformations in train mode
    testing_instances = instance_sampler.apply(transformed_data, is_train=True)

    return as_stacked_batches(
        testing_instances,
        batch_size=batch_size,
        output_type=torch.tensor,
        field_names=PREDICTION_INPUT_NAMES,
    )

We have a test dataloader helper for completion, even though we will not use it here. This is useful in a production setting where we want to start forecasting from the end of a given time series. Thus, the test dataloader will sample the very last context window from the dataset provided and pass it to the model.

In [212]:
def create_test_dataloader(
    config: PretrainedConfig,
    freq,
    data,
    batch_size: int,
    **kwargs,
):
    PREDICTION_INPUT_NAMES = [
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data, is_train=False)

    # We create a test Instance splitter to sample the very last
    # context window from the dataset provided.
    instance_sampler = create_instance_splitter(config, "test")

    # We apply the transformations in test mode
    testing_instances = instance_sampler.apply(transformed_data, is_train=False)

    return as_stacked_batches(
        testing_instances,
        batch_size=batch_size,
        output_type=torch.tensor,
        field_names=PREDICTION_INPUT_NAMES,
    )

In [213]:
train_dataloader = create_train_dataloader(
    config=config,
    freq=freq,
    data=train_dataset,
    batch_size=256,
    num_batches_per_epoch=100,
)
train_dataloader_iter = iter(train_dataloader)
print(type(train_dataloader), train_dataloader.length, dir(train_dataloader))

custom_back_test_sampler = CustomBackTestSampler(validation_length)
print(type(custom_back_test_sampler), custom_back_test_sampler)
test_dataloader = create_backtest_dataloader(
    config=config,
    freq=freq,
    data=val_data,
    batch_size=256,
    validation_sampler=custom_back_test_sampler,
)
test_dataloader_iter = iter(test_dataloader)
print(type(test_dataloader), test_dataloader.length, dir(test_dataloader))

<class 'gluonts.itertools.IterableSlice'> 100 ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'iterable', 'length']
<class '__main__.CustomBackTestSampler'> axis=-1 min_past=0 min_future=0 back_test_size=1100
<class 'gluonts.itertools.IterableSlice'> None ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'iterable', 'length']


Let's check the first batch:

In [214]:
prev_time = time.time()
batch = next(train_dataloader_iter)
for k, v in batch.items():
    print(k, v.shape, v.type())
curr_time = time.time()
print(curr_time - prev_time)

past_time_features torch.Size([256, 27, 25]) torch.FloatTensor
past_values torch.Size([256, 27]) torch.FloatTensor
past_observed_mask torch.Size([256, 27]) torch.FloatTensor
future_time_features torch.Size([256, 1, 25]) torch.FloatTensor
static_categorical_features torch.Size([256]) torch.LongTensor
future_values torch.Size([256, 1]) torch.FloatTensor
future_observed_mask torch.Size([256, 1]) torch.FloatTensor
1.031831979751587


In [215]:
prev_time = time.time()
test_sample_batch = next(test_dataloader_iter)
for k, v in test_sample_batch.items():
    print(k, v.shape, v.type())
curr_time = time.time()
print(curr_time - prev_time)

past_time_features torch.Size([256, 27, 25]) torch.FloatTensor
past_values torch.Size([256, 27]) torch.FloatTensor
past_observed_mask torch.Size([256, 27]) torch.FloatTensor
future_time_features torch.Size([256, 1, 25]) torch.FloatTensor
static_categorical_features torch.Size([256]) torch.LongTensor
0.012508630752563477


In [234]:
# verify that custom_back_test_sampler and val_data are correct
def verify_print_sample(sample_batch, sample_idx):
    print("past_time_features", sample_batch["past_time_features"][sample_idx, -2:, 0:2])
    print("past_values", sample_batch["past_values"][sample_idx])
    print("future_time_features", sample_batch["future_time_features"][sample_idx])
    print("static_categorical_features", sample_batch["static_categorical_features"][sample_idx])
test_dataloader_iter_verify = iter(test_dataloader)
test_sample_batch_verify = next(test_dataloader_iter_verify)
for k, v in test_sample_batch_verify.items():
    print(k, v.shape, v.type())
verify_print_sample(test_sample_batch_verify, 0)
verify_print_sample(test_sample_batch_verify, 1)
verify_print_sample(test_sample_batch_verify, 2)
display(df[df["stock_id"] == 0][["time_id", "target", 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag']].iloc[- 55 * 20 - 7:- 55 * 20 + 3])

past_time_features torch.Size([256, 27, 25]) torch.FloatTensor
past_values torch.Size([256, 27]) torch.FloatTensor
past_observed_mask torch.Size([256, 27]) torch.FloatTensor
future_time_features torch.Size([256, 1, 25]) torch.FloatTensor
static_categorical_features torch.Size([256]) torch.LongTensor
past_time_features tensor([[5.3000e+02, 3.3364e+06],
        [5.4000e+02, 3.3364e+06]])
past_values tensor([ 2.9302, -1.6803, -0.9799, -3.6198, -5.4997, -2.2298, -3.6597, -1.6701,
        -1.9199, -2.1702, -1.3101,  2.2101,  4.2105,  4.8399,  6.5804,  4.8494,
         5.2595,  3.1698,  0.9406,  1.4699, -1.8901, -1.0598, -0.5400, -4.8298,
        -4.5800, -6.7401,  0.6104])
future_time_features tensor([[ 0.0000e+00,  1.0247e+06,  1.0000e+00,  1.0003e+00,  1.5223e+07,
          0.0000e+00,  0.0000e+00,  9.9999e-01,  1.3050e+03,  1.0003e+00,
          6.5927e+04,  1.0000e+00,  2.5355e+04, -9.6118e-01, -8.7386e-01,
          0.0000e+00,  0.0000e+00,  4.8313e+01,  4.8313e+01,  1.5543e+01,
      

,time_id,target,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag
timestamp,,,,,
2018-01-18 14:28,25348.0,-1.890063,480.0,3317490.250,1.0
2018-01-18 14:29,25349.0,-1.059771,490.0,3923400.750,1.0
2018-01-18 14:30,25350.0,-0.540018,500.0,3485172.250,1.0
2018-01-18 14:31,25351.0,-4.829764,510.0,3485172.250,1.0
2018-01-18 14:32,25352.0,-4.580021,520.0,3337968.250,1.0
2018-01-18 14:33,25353.0,-6.740093,530.0,3336365.500,1.0
2018-01-18 14:34,25354.0,0.610352,540.0,3336365.500,1.0
2018-01-18 14:35,25355.0,2.900362,0.0,1024700.375,1.0
2018-01-18 14:36,25356.0,8.339882,10.0,1799780.625,1.0


As can be seen, we don't feed `input_ids` and `attention_mask` to the encoder (as would be the case for NLP models), but rather `past_values`, along with `past_observed_mask`, `past_time_features`, and `static_categorical_features`.

The decoder inputs consist of `future_values`, `future_observed_mask` and `future_time_features`. The `future_values` can be seen as the equivalent of `decoder_input_ids` in NLP.

We refer to the [docs](https://huggingface.co/docs/transformers/model_doc/time_series_transformer#transformers.TimeSeriesTransformerForPrediction.forward.past_values) for a detailed explanation for each of them.

## Forward pass

Let's perform a single forward pass with the batch we just created:

In [235]:
# perform forward pass
outputs = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"]
    if config.num_static_categorical_features > 0
    else None,
    static_real_features=batch["static_real_features"]
    if config.num_static_real_features > 0
    else None,
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
    future_observed_mask=batch["future_observed_mask"],
    output_hidden_states=True,
)

In [236]:
print("Loss:", outputs.loss.item())

Loss: 3.532121419906616


Note that the model is returning a loss. This is possible as the decoder automatically shifts the `future_values` one position to the right in order to have the labels. This allows computing a loss between the predicted values and the labels.

Also note that the decoder uses a causal mask to not look into the future as the values it needs to predict are in the `future_values` tensor.

## Train the Model

It's time to train the model! We'll use a standard PyTorch training loop.

We will use the 🤗 [Accelerate](https://huggingface.co/docs/accelerate/index) library here, which automatically places the model, optimizer and dataloader on the appropriate `device`.

In [237]:
from accelerate import Accelerator
from torch.optim import AdamW

accelerator = Accelerator()
device = accelerator.device
print(device)

cuda


In [238]:
model.to(device)
optimizer = AdamW(model.parameters(), lr=6e-4, betas=(0.9, 0.95), weight_decay=1e-1)

In [239]:
model, optimizer, train_dataloader = accelerator.prepare(
    model,
    optimizer,
    train_dataloader,
)

In [240]:
epochs = 1

In [241]:
prev_time = time.time()
val_set_parsed_data = list(iter(val_data))
# TODO: do not use offset split -1
val_set_ground_truth = list(map(lambda stock_data: stock_data["target"][-validation_length + 1:], val_set_parsed_data))
val_set_ground_truth = np.vstack(val_set_ground_truth).flatten()
curr_time = time.time()
print(val_set_ground_truth.shape, curr_time - prev_time)

(219800,) 0.8182122707366943


In [242]:
def eval_epoch(model):
    model.eval()

    forecasts = []

    for idx, batch in enumerate(test_dataloader):
        outputs = model.generate(
            static_categorical_features=batch["static_categorical_features"].to(device)
            if config.num_static_categorical_features > 0
            else None,
            static_real_features=batch["static_real_features"].to(device)
            if config.num_static_real_features > 0
            else None,
            past_time_features=batch["past_time_features"].to(device),
            past_values=batch["past_values"].to(device),
            future_time_features=batch["future_time_features"].to(device),
            past_observed_mask=batch["past_observed_mask"].to(device),
        )
        forecasts.append(outputs.sequences.cpu().numpy())
    
    # print(forecasts[0].shape)
    forecasts = np.vstack(forecasts)
    # print(forecasts.shape)

    forecast_median = np.median(forecasts, 1)
    mae = mae_metric.compute(predictions=forecast_median,references=val_set_ground_truth)    
    return mae, forecasts

##### Initial MAE (random weights from model initialization)

In [243]:
prev_time = time.time()
result = eval_epoch(model)
curr_time = time.time()
print(result[0], curr_time - prev_time)

{'mae': 5.859654777101835} 40.93197536468506


In [74]:
for epoch in range(epochs):
    epoch_start_time = time.time()

    model.train()
    
    for idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(
            static_categorical_features=batch["static_categorical_features"].to(device)
            if config.num_static_categorical_features > 0 
            else None,
            static_real_features=batch["static_real_features"].to(device)
            if config.num_static_real_features > 0
            else None,
            past_time_features=batch["past_time_features"].to(device),
            past_values=batch["past_values"].to(device),
            future_time_features=batch["future_time_features"].to(device),
            future_values=batch["future_values"].to(device),
            past_observed_mask=batch["past_observed_mask"].to(device),
            future_observed_mask=batch["future_observed_mask"].to(device),
        )
        loss = outputs.loss

        # Backpropagation
        accelerator.backward(loss)
        optimizer.step()

    epoch_train_end_time = time.time()

    mae = eval_epoch(model)
    epoch_eval_end_time = time.time()

    print(f"epoch: {epoch} - mae: {mae}, loss: {loss.item()}, epoch train time: {epoch_train_end_time - epoch_start_time}, eval time: {epoch_eval_end_time - epoch_train_end_time}")

NameError: name 'eval_epoch' is not defined

## Inference

At inference time, it's recommended to use the `generate()` method for autoregressive generation, similar to NLP models.

Forecasting involves getting data from the test instance sampler, which will sample the very last `context_length` sized window of values from each time series in the dataset, and pass it to the model. Note that we pass `future_time_features`, which are known ahead of time, to the decoder.

The model will autoregressively sample a certain number of values from the predicted distribution and pass them back to the decoder to return the prediction outputs:

In [ ]:
model.eval()

forecasts = []

for batch in test_dataloader:
    outputs = model.generate(
        static_categorical_features=batch["static_categorical_features"].to(device)
        if config.num_static_categorical_features > 0
        else None,
        static_real_features=batch["static_real_features"].to(device)
        if config.num_static_real_features > 0
        else None,
        past_time_features=batch["past_time_features"].to(device),
        past_values=batch["past_values"].to(device),
        future_time_features=batch["future_time_features"].to(device),
        past_observed_mask=batch["past_observed_mask"].to(device),
    )
    forecasts.append(outputs.sequences.cpu().numpy())

The model outputs a tensor of shape (`batch_size`, `number of samples`, `prediction length`).

In this case, we get `100` possible values for the next `24` months (for each example in the batch which is of size `64`):

In [ ]:
forecasts[0].shape

We'll stack them vertically, to get forecasts for all time-series in the test dataset:

In [ ]:
forecasts = np.vstack(forecasts)
print(forecasts.shape)

We can evaluate the resulting forecast with respect to the ground truth out of sample values present in the test set. For that, we'll use the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library, which includes the [MASE](https://huggingface.co/spaces/evaluate-metric/mase) and [sMAPE](https://huggingface.co/spaces/evaluate-metric/smape) metrics.

We calculate both metrics for each time series in the dataset:

In [ ]:
from evaluate import load
from gluonts.time_feature import get_seasonality

mae_metric = load("evaluate-metric/mae")

In [ ]:
forecast_median = np.median(forecasts, 1)

In [ ]:
ground_truth = [test_data['target'][-prediction_length:] for test_data in test_dataset]

In [ ]:
mae = mae_metric.compute(predictions=forecast_median,references=ground_truth)

In [ ]:
print(f"MAE: {mae}")

To plot the prediction for any time series with respect the ground truth test data we define the following helper:

In [ ]:
import matplotlib.dates as mdates


def plot(ts_index):
    fig, ax = plt.subplots()

    index = pd.period_range(
        start=test_dataset[ts_index][FieldName.START],
        periods=len(test_dataset[ts_index][FieldName.TARGET]),
        freq=freq,
    ).to_timestamp()

    # Major ticks every half year, minor ticks every month,
    ax.xaxis.set_major_locator(mdates.MinuteLocator())
    ax.xaxis.set_minor_locator(mdates.MinuteLocator())

    ax.plot(
        index[-2 * prediction_length :],
        test_dataset[ts_index]["target"][-2 * prediction_length :],
        label="actual",
    )

    plt.plot(
        index[-prediction_length:],
        np.median(forecasts[ts_index], axis=0),
        label="median",
    )

    plt.fill_between(
        index[-prediction_length:],
        forecasts[ts_index].mean(0) - forecasts[ts_index].std(axis=0),
        forecasts[ts_index].mean(0) + forecasts[ts_index].std(axis=0),
        alpha=0.3,
        interpolate=True,
        label="+/- 1-std",
    )
    plt.legend()
    plt.show()

For example:

In [ ]:
plot(0)

How do we compare against other models? The [Monash Time Series Repository](https://forecastingdata.org/#results) has a comparison table of test set MASE metrics which we can add to:

|Dataset | 	SES| 	Theta | 	TBATS| 	ETS	| (DHR-)ARIMA| 	PR|	CatBoost |	FFNN	| DeepAR | 	N-BEATS | 	WaveNet| 	**Transformer** (Our) |
|:------------------:|:-----------------:|:--:|:--:|:--:|:--:|:--:|:--:|:---:|:---:|:--:|:--:|:--:|
|Tourism Monthly | 	3.306 |	1.649 |	1.751 |	1.526|	1.589|	1.678	|1.699|	1.582	| 1.409	| 1.574|	1.482	|  **1.256**|

Note that, with our model, we are beating all other models reported (see also table 2 in the corresponding [paper](https://openreview.net/pdf?id=wEc1mgAjU-)), and we didn't do any hyperparameter tuning. We just trained the Transformer for 40 epochs.

Of course, we need to be careful with just claiming state-of-the-art results on time series with neural networks, as it seems ["XGBoost is typically all you need"](https://www.sciencedirect.com/science/article/pii/S0169207021001679).  We are just very curious to see how far neural networks can bring us, and whether Transformers are going to be useful in this domain. This particular dataset seems to indicate that it's definitely worth exploring.

## Next Steps

We would encourage the readers to try out the [notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/time-series-transformers.ipynb) with other time series datasets from the [Hub](https://huggingface.co/datasets/monash_tsf) and replace the appropriate frequency and prediction length parameters. For your datasets, one would need to convert them to the convention used by GluonTS, which is explained nicely in their documentation [here](https://ts.gluon.ai/stable/tutorials/forecasting/extended_tutorial.html#What-is-in-a-dataset?). We have also prepared an example notebook showing you how to convert your dataset into the 🤗 datasets format [here](https://github.com/huggingface/notebooks/blob/main/examples/time_series_datasets.ipynb).

As time series researchers will know, there has been a lot of interest in applying Transformer based models to the time series problem. The vanilla Transformer is just one of many attention-based models and so there is a need to add more models to the library.

At the moment there is nothing stopping us from modeling multivariate time series, however for that one would need to instantiate the model with a multivariate distribution head. Currently, diagonal independent distributions are supported, and other multivariate distributions will be added. Stay tuned for a future blog post which will include a tutorial.

Another thing on the roadmap is time series classification. This entails adding a time series model with a classification head to the library, for the anomaly detection task for example.

The current model assumes the presence of a date-time together with the time series values, which might not be the case for every time series in the wild. See for instance neuroscience datasets like the one from [WOODS](https://woods-benchmarks.github.io/). Thus, one would need to generalize the current model to make some inputs optional in the whole pipeline.

Finally, the NLP/Vision domain has benefitted tremendously from [large pre-trained models](https://arxiv.org/abs/1810.04805), while this is not the case as far as we are aware for the time series domain. Transformer based models seem like the obvious choice in pursuing this avenue of research and we cannot wait to see what researchers and practitioners come up with!
